# 시맨틱 검색 - Semantic Search


In [1]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3

## 사전준비


### 데이터 준비 및 탐색


In [2]:
import pandas as pd
import requests

df = pd.read_csv("./data/movies.csv", low_memory=False)
df.head(5)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,1917,드라마|전쟁,2020,2.19,8.88,5017,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,1.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...
2,n번째 이별중,코미디|멜로/로맨스,2020,4.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...
3,가슴 큰 태희,멜로/로맨스,2020,2.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,정보없음,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...
4,개인교습,멜로/로맨스,2020,3.02,10.00,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11947 entries, 0 to 11946
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       11947 non-null  object 
 1   genre       11947 non-null  object 
 2   year        11947 non-null  int64  
 3   date        11947 non-null  float64
 4   rating      11947 non-null  float64
 5   vote_count  11947 non-null  int64  
 6   plot        11947 non-null  object 
 7   main_act    11947 non-null  object 
 8   supp_act    11947 non-null  object 
 9   page_url    11947 non-null  object 
 10  img_url     11947 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.0+ MB


In [4]:
df.rename(columns={"plot": "text"}, inplace=True)
df.head(5)

,title,genre,year,date,rating,vote_count,text,main_act,supp_act,page_url,img_url
0,1917,드라마|전쟁,2020,2.19,8.88,5017,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,1.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...
2,n번째 이별중,코미디|멜로/로맨스,2020,4.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...
3,가슴 큰 태희,멜로/로맨스,2020,2.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,정보없음,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...
4,개인교습,멜로/로맨스,2020,3.02,10.00,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...


### OpenSearch 도메인에 연결


In [5]:
# Add system path for utils
import sys

sys.path.insert(0, "/Users/jinhwan/Repository/Labs/opensearch-with-bedrock-workshop-kr/utils")

In [6]:
import boto3, json
from utils import get_cfn_outputs

region_name = "us-east-1"

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]

In [7]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

## 데이터 임베딩을 위한 Ingest Pipeline 생성


모델 아이디 설정


In [8]:
model_id = "jK-l048BhWoFUAg8WOdO"

인제스트 파이프라인 생성


In [9]:
pipeline = {
    "description": "An neural search pipeline for hotel index - OpenSearch-cohere-060124084807",
    "processors": [
        {
            "text_embedding": {
                "model_id": model_id,
                "field_map": {
                    "text": "vector_field",
                },
            }
        }
    ],
}

pipeline_id = "webtoon_embedding_pipeline"
aos_client.ingest.put_pipeline(id=pipeline_id, body=pipeline)

{'acknowledged': True}

인덱스 생성


In [10]:
index_name = "movie_semantic"

movie_semantic = {
    "settings": {
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
        "index.knn": True,
        "default_pipeline": pipeline_id,
        "index.knn.space_type": "cosinesimil",
    },
    "mappings": {
        "properties": {
            "date": {
                "type": "float",
            },
            "genre": {
                "type": "text",
            },
            "img_url": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "main_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "page_url": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "text": {
                "type": "text",
            },
            "rating": {"type": "float"},
            "supp_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "title": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vote_count": {"type": "long"},
            "year": {"type": "long"},
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },
        }
    },
}

aos_client.indices.create(index=index_name, body=movie_semantic)

RequestError: RequestError(400, 'resource_already_exists_exception', 'index [movie_semantic/3YlpqEWlRwqvF6lMa4SgAQ] already exists')

데이터 인제스트


In [ ]:
from tqdm import tqdm
from opensearchpy import helpers

json_data = df.to_json(orient="records", lines=True)
docs = json_data.split("\n")[:-1]  # To remove the last empty line


def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}


succeeded = []
failed = []
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=20, thread_count=2, queue_size=2
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

잘 저장되었는지 확인


In [ ]:
# Refresh the index to make the changes visible
aos_client.indices.refresh(index=index_name)

count = aos_client.count(index=index_name)
print(count)

검색 비교


In [ ]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["text"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_id"],
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["text"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_id", "_score", "title", "plot", "genre", "rating"]
    )
    display(query_result_df)

In [ ]:
query_text = "슈퍼맨"

In [ ]:
keyword_search(query_text)

In [ ]:
def semantic_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["vector_field"]},
        "query": {
            "neural": {"vector_field": {"query_text": query_text, "model_id": model_id, "k": 30}},
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_id"],
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["text"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_id", "_score", "title", "plot", "genre", "rating"]
    )
    display(query_result_df)

In [ ]:
semantic_search(query_text)

## Clean Up


In [11]:
# aos_client.indices.delete(index=index_name)

{'acknowledged': True}